# RQ3 Questions

In [1]:
import json
import pandas as pd
from tweepy import Cursor
from tweepy import API
import numpy as np
import matplotlib.pyplot as plt
from tweepy import OAuthHandler
import networkx as nx
import igraph
from scipy.sparse import csr_matrix
import collections
from collections import defaultdict

In [2]:
with open("data/processed_retweets.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

In [3]:
df_raw_tweets = pd.DataFrame.from_records(data)

In [4]:
df_retweets = df_raw_tweets[df_raw_tweets["text"].apply(lambda x: x[:2]) == "RT"]

In [5]:
def createRetweetsGraph(df_retweets):
    g = igraph.Graph()
    users = []
    edges = []
    user2ret_user = {}
    for index, row in df_retweets.iterrows():
        user = row["user"]["screen_name"]
        if(user not in users):
            users.append(user)
        mentioned_users = list(filter(lambda word: word[0]=='@', row['text'].split()))
        if(len(mentioned_users)>0):
            ret_user = mentioned_users[0][1:-1]  
            if(ret_user not in users):
                users.append(ret_user)  
            edges.append((user, ret_user))
    g.add_vertices(users)
    g.add_edges(edges)
    return g

In [65]:
original_graph = createRetweetsGraph(df_retweets)


In [66]:
original_graph.vcount()

40092

In [67]:
original_graph.ecount()

34169

## Test and train data

In [20]:
def find_nodes_at_distance_2(graph):
    all_potential_recommendations = set()
    
    for n1 in graph.vs:

        nodes_at_most_distant_1 = set(graph.neighborhood(vertices=n1, order=1))
        nodes_at_most_distant_2 = set(graph.neighborhood(vertices=n1, order=2))

        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        if len(only_nodes_at_distance_2) > 0:
        
            for n2 in only_nodes_at_distance_2:
                n1_index = n1.index
                
                all_potential_recommendations.add((n1_index, n2))
            
    return all_potential_recommendations

In [21]:
nodes_distance_2 = find_nodes_at_distance_2(original_graph)

In [22]:
# fraction of edges to select as test-set
p = 0.2

# graphsize
N = len(original_graph.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a=all_idxs, size=int(p*N), replace=False)

D2 = np.random.permutation(list(nodes_distance_2))[:(len(test_idxs))]

In [23]:
len(test_idxs)

1315

In [24]:
ground_truth = set()
for idx, one_edge in enumerate(original_graph.es):
    n1 = one_edge.source
    n2 = one_edge.target

    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
for n1, n2 in D2:
    ground_truth.add((n1, n2, 0))

## ALS

In [262]:
import implicit


In [270]:
# first we get the adjacency matrix data
M = original_graph.get_adjacency().data
M = csr_matrix(M)

In [271]:
model = implicit.als.AlternatingLeastSquares(factors=10, calculate_training_loss=True,  iterations=5)
# train the model
model.fit(M)

100%|██████████| 5/5 [00:02<00:00,  2.04it/s, loss=0.00015]


In [272]:
def predict_ALS(testset, model):
    
    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1,n2, w in testset:
        
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [273]:
# generate the predictions
df_test = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])
all_predictions = predict_ALS(df_test.values, model)

# add predictions to df
df_test["rating"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test["rating"] = df_test["rating"].apply(lambda x: round(x))

In [275]:
# number of observations matched by the prediction
right_predictions = len(df_test[df_test["rating"] == df_test["edge"]])

# accuracy
right_predictions/len(df_test)

0.5731857318573186

## ADA

In [281]:
def compute_ADA(u,v, graph):
    
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    
    # set Z of neighbors of both
    bridges = set(outlinks_from_u).intersection(inlinks_to_v)

    # degree of nodes in set Z
    deg_ = [graph.degree(n) for n in bridges]
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

In [287]:
ADA_test = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])

In [288]:
allscores= []
for n1,n2, w in ADA_test.values:
    score = compute_ADA(n1,n2,original_graph)
    allscores.append(score)
ADA_test["rating"] = allscores
ADA_test["rating"] = ADA_test["rating"].apply(lambda x: round(x))

In [290]:
right_predictions = len(ADA_test[ADA_test["rating"] == ADA_test["edge"]])

# accuracy
right_predictions/len(ADA_test)

0.5014350143501435

## Page rank

In [304]:
PR_test = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])

In [307]:
pagerank = enumerate(original_graph.personalized_pagerank(reset_vertices=10))
allpredictions= []
for n1,n2, w in PR_test.values:
    pagerank = enumerate(original_graph.personalized_pagerank(reset_vertices=n1))
    for i, val in pagerank:
        if(i == n2):
            allpredictions.append(val)
            break


In [309]:
PR_test["rating"] = allpredictions
PR_test["rating"] = PR_test["rating"].apply(lambda x: round(x))

In [311]:
right_predictions = len(PR_test[PR_test["rating"] == PR_test["edge"]])

# accuracy
right_predictions/len(PR_test)

0.5038950389503895

# Proposed algorithm

## Logistic regresion

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [48]:
# fraction of edges to select as test-set
p = 0.2
# graphsize
N = len(original_graph.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a=all_idxs, size=int(p*N), replace=False)
#train idx of edges
train_idx = []
for i in range(N):
    if(i not in test_idxs):
        train_idx.append(i)
D2 = np.random.permutation(list(nodes_distance_2))[:(len(test_idxs))]

In [ ]:
ground_truth = set()
for idx, one_edge in enumerate(original_graph.es):
    n1 = one_edge.source
    n2 = one_edge.target

    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
for n1, n2 in D2:
    ground_truth.add((n1, n2, 0))

In [68]:
trainX = []
trainY = []
for u,v,w in ground_truth:
    trainX.append((u,v))
    trainY.append(w)

In [50]:
trainX = []
trainY = []
for idx, one_edge in enumerate(original_graph.es):
    n1 = one_edge.source
    n2 = one_edge.target
    link = 1
    if idx in test_idxs:
        trainX.append((n1, n2))
        trainY.append(0)
    else:
        trainX.append((n1, n2))
        trainY.append(1)

In [31]:
LR_test = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])

In [52]:
testX = []
testY = []
for u,v,w in ground_truth:
    testX.append((u,v))
    testY.append(w)

In [9]:
M = original_graph.get_adjacency().data
M = csr_matrix(M)

In [51]:
lr = LogisticRegression(class_weight="balanced")
lr.fit(trainX,trainY )

LogisticRegression(class_weight='balanced')

In [53]:
allpredictions = lr.predict_proba(testX)

In [57]:
LR_test["rating"] = allpredictions[:,1]
LR_test["rating"] = LR_test["rating"].apply(lambda x: round(x))

In [61]:
right_predictions = len(LR_test[LR_test["rating"] == LR_test["edge"]])
# accuracy
right_predictions/len(LR_test)

0.694358748845214
